<a href="https://colab.research.google.com/github/PrestoshTJ/QualifAI/blob/main/webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymongo

In [ ]:
# Created With help from https://www.youtube.com/watch?v=-H-JCgvV0z8
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
from collections import deque
import time
import re
import json
import pymongo



keywords = {"Computer%20Science", "Engineering", "Business%20Administration", "Marketing", "Graphic%20Design", "Data%20Science", "Information%20Technology", "Communications", "Finance", "Environmental%20Science"}

In [ ]:
id_list = {keyword: [] for keyword in keywords}




for keyword in keywords:
  q = deque()
  for i in range(0, 51, 10):
    q.append(i)
  while q:
    i = q.popleft()
    list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={keyword}&start={i}"

    response = requests.get(list_url)
    if not response.status_code == 200:
        time.sleep(2)
        q.append(i)
        continue
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")

    for job in page_jobs:
      base_card_div = job.find("div", {"class": "base-card"})
      job_id = base_card_div.get("data-entity-urn").split(":")[3]
      id_list[keyword].append(job_id)
  print(len(id_list[keyword]))


In [ ]:
print(len(id_list))
job_list = []

In [ ]:
q = deque()
for keyword in id_list:
  word = " ".join(keyword.split("%20"))
  for job_id in id_list[keyword]:
    q.append((job_id, word))

In [ ]:
while q:
  job = q.popleft()
  job_id = job[0]
  keyword = job[1]
  job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
  job_response = requests.get(job_url)
  job_soup = BeautifulSoup(job_response.text, "html.parser")
  job_post = {}
  if not job_response.status_code == 200:
    q.append((job_id, keyword))
    time.sleep(2)
    continue
  job_post["job_id"] = job_id
  job_post["keyword"] = keyword
  try:
    job_post["job_title"] = job_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
  except:
    job_post["job_title"] = None
  try:
    job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
  except:
    job_post["company_name"] = None
  try:
    job_post["time_posted"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
  except:
    job_post["time_posted"] = None
  try:
    job_post["num_applicants"] = job_soup.find("span", {"class": "num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip();
  except:
      try:
        job_post["num_applicants"] = job_soup.find("figcaption", {"class": "num-applicants__caption"}).text.strip();
      except:
        job_post["num_applicants"] = None
  try:
    job_post["location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip()
  except:
    job_post["location"] = None
  try:
    job_post["experience_level"] = job_soup.find_all("span", {"class": "description__job-criteria-text description__job-criteria-text--criteria"})[0].text.strip()
  except:
    job_post["experience_level"] = None
  try:
    job_post["employment_type"] = job_soup.find_all("span", {"class": "description__job-criteria-text description__job-criteria-text--criteria"})[1].text.strip()
  except:
    job_post["employment_type"] = None

  print(len(q))
  print(len(job_list))
  job_list.append(job_post)

In [ ]:
print(len(q))
print(len(job_list))
print(job_list)

In [ ]:
final = json.dumps(job_list, indent=2)


In [ ]:
with open('Internships.json', 'w') as f:
  f.write(final)